In [ ]:
p_center = "Yale"
p_ROI = "CC400"

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
pip install torch-geometric

In [ ]:
import pandas as pd
# 1 is for asd and 0 is for healthy
df_labels = pd.read_csv('/content/drive/My Drive/Phenotypic_V1_0b_preprocessed1.csv')#path
df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})


labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label


In [ ]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3])
    else:
        key = '_'.join(f_split[0:2])
    return key

In [ ]:
import os
data_main_path = '/content/drive/My Drive/ccprecdata/ABIDE_pcp/cpac/filt_global' #path to time series data
#data_main_path = '/content/drive/My Drive/power264'
flist = os.listdir(data_main_path)
print(len(flist))

for f in range(len(flist)):
    flist[f] = get_key(flist[f])

In [ ]:
import numpy as np
centers_dict = {}
for f in flist:
    key = f.split('_')[0]

    if key not in centers_dict:
        centers_dict[key] = []
    centers_dict[key].append(f)

flist = np.array(centers_dict[p_center])

In [ ]:
ASD_labels = []
for f in flist:
    ASD_labels.append(labels[f])

print(len(ASD_labels))

In [ ]:
folder_path = '/content/drive/My Drive/ccprecdata/ABIDE_pcp/cpac/filt_global'
fMRI_samples = []
selected_files = [f for f in os.listdir(folder_path) if f.startswith('Yale')]
print(selected_files)
for file_name in selected_files:
    file_path = os.path.join(folder_path, file_name)
    data = np.loadtxt(file_path)
    print(data.shape)
    fMRI_samples.append(data)

time_series = np.array(fMRI_samples)
print(time_series.shape)

In [ ]:
selected_indices = [1, 2, 17, 54, 57, 108, 147, 173]
time_series = time_series[:, :, selected_indices]

### implemeting GraphSAGE

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from scipy.fft import fft

n_samples = 41   #samples (subjects)
n_regions = 8   #brain regions
n_time_points = 196  #time points in each fMRI time series

# Function to compute node features using Fourier Transform
def compute_frequency_domain_features(time_series):
    fft_features = np.abs(fft(time_series, axis=-1))
    return fft_features[:, :n_time_points // 2]

# Compute Fourier Transform features for each subject (node features)
node_features_list = [compute_frequency_domain_features(time_series[i]) for i in range(n_samples)]

# Function to compute correlation matrix (used as edges)
def compute_correlation_matrix(time_series):
    corr_matrix = np.corrcoef(time_series.T)
    return corr_matrix


correlation_matrices = [compute_correlation_matrix(time_series[i]) for i in range(n_samples)]

# Build graph data for each subject
graph_data_list = []
for i in range(n_samples):
    x = torch.tensor(node_features_list[i], dtype=torch.float)

    corr_matrix = correlation_matrices[i]
    edge_index = np.array(np.where(corr_matrix > 0))  # Edge threshold
    edge_index = torch.tensor(edge_index, dtype=torch.long)

    num_nodes = x.size(0)
    edge_index = edge_index[:, (edge_index[0] < num_nodes) & (edge_index[1] < num_nodes)]

    # Create PyG Data object for the sample
    data = Data(x=x, edge_index=edge_index)
    graph_data_list.append(data)


# Train-test split
train_idx, test_idx = train_test_split(np.arange(n_samples), test_size=0.1, random_state=10)

# Ensure train_idx and test_idx are lists of integers
train_idx = train_idx.tolist() if isinstance(train_idx, torch.Tensor) else list(train_idx)
test_idx = test_idx.tolist() if isinstance(test_idx, torch.Tensor) else list(test_idx)

train_data = [graph_data_list[i] for i in train_idx]

# Index the labels
train_labels = torch.tensor([ASD_labels[i] for i in train_idx], dtype=torch.long)
test_labels = torch.tensor([ASD_labels[i] for i in test_idx], dtype=torch.long)

test_data = [graph_data_list[i] for i in test_idx]

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the GraphSAGE model
class GraphSAGE(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(num_node_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        return F.log_softmax(x, dim=1)


model = GraphSAGE(num_node_features=node_features_list[0].shape[1], hidden_channels=128, num_classes=2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train():
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()

        out = model(batch)
        batch_labels = train_labels[batch.batch]

        loss = F.nll_loss(out, batch_labels)
        loss.backward()
        optimizer.step()

def test(loader, labels):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:

            out = model(batch)

            pred = out.argmax(dim=1)
            #print(f'Pred shape: {pred.shape}')

            # Map batch indices to the corresponding labels
            batch_indices = batch.batch
            batch_labels = labels[batch_indices]

            # Compute the number of correct predictions
            correct += (pred == batch_labels).sum().item()
            total += batch_labels.size(0)

    accuracy = correct / total if total > 0 else 0
    return accuracy * 100  # Convert to percentage

# Training loop
for epoch in range(100):
    train()
    #train_acc = test(train_loader, train_labels)
    test_acc = test(test_loader, test_labels)
    print(f'Epoch: {epoch:03d}, Test Acc: {test_acc:.4f}')


### implementing GCN

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from scipy.fft import fft
from scipy.spatial.distance import pdist, squareform

n_samples = 172   # samples (subjects)
n_regions = 392   #brain regions
n_time_points = 176  # time points in each fMRI time series


# Function to compute node features using Fourier Transform
def compute_frequency_domain_features(time_series):
    fft_features = np.abs(fft(time_series, axis=-1))
    return fft_features[:, :n_time_points // 2]

# Compute Fourier Transform features for each subject (node features)
node_features_list = [compute_frequency_domain_features(time_series[i]) for i in range(n_samples)]

# Function to compute correlation matrix (used as edges)
def compute_correlation_matrix(time_series):
    corr_matrix = np.corrcoef(time_series.T)  # Correlation matrix for brain regions
    return corr_matrix

correlation_matrices = [compute_correlation_matrix(time_series[i]) for i in range(n_samples)]

# Build graph data for each subject
graph_data_list = []
for i in range(n_samples):

    x = torch.tensor(node_features_list[i], dtype=torch.float)

    # Correlation matrix (edges)
    corr_matrix = correlation_matrices[i]
    edge_index = np.array(np.where(corr_matrix > 1.5))  # Edge threshold
    edge_index = torch.tensor(edge_index, dtype=torch.long)

    # Ensure valid edge indices
    num_nodes = x.size(0)
    edge_index = edge_index[:, (edge_index[0] < num_nodes) & (edge_index[1] < num_nodes)]

    # Create PyG Data object for the sample
    data = Data(x=x, edge_index=edge_index)
    graph_data_list.append(data)


train_idx, test_idx = train_test_split(np.arange(n_samples), test_size=0.2, random_state=0)

# Ensure train_idx and test_idx are lists of integers
train_idx = train_idx.tolist() if isinstance(train_idx, torch.Tensor) else list(train_idx)
test_idx = test_idx.tolist() if isinstance(test_idx, torch.Tensor) else list(test_idx)

train_data = [graph_data_list[i] for i in train_idx]

# Index the labels
train_labels = torch.tensor([ASD_labels[i] for i in train_idx], dtype=torch.long)
test_labels = torch.tensor([ASD_labels[i] for i in test_idx], dtype=torch.long)

test_data = [graph_data_list[i] for i in test_idx]

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(num_node_features=node_features_list[0].shape[1], hidden_channels=512, num_classes=2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train():
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()

        out = model(batch)
        print(f'Output shape: {out.shape}')

        batch_labels = train_labels[batch.batch]
        print(f'Batch labels shape: {batch_labels.shape}')

        loss = F.nll_loss(out, batch_labels)
        loss.backward()
        optimizer.step()

def test(loader, labels):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            out = model(batch)
            print(f'Output shape: {out.shape}')

            pred = out.argmax(dim=1)
            print(f'Pred shape: {pred.shape}')

            # map batch indices to the corresponding labels
            batch_indices = batch.batch
            batch_labels = labels[batch_indices]

            # Compute the number of correct predictions
            correct += (pred == batch_labels).sum().item()
            total += batch_labels.size(0)

    accuracy = correct / total if total > 0 else 0
    return accuracy * 100  # Convert to percentage

# Training loop
for epoch in range(100):
    train()
    train_acc = test(train_loader, train_labels)
    test_acc = test(test_loader, test_labels)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


### implementing SparseGAT

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv  # Import GATConv for SparseGAT
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from scipy.fft import fft

n_samples = 41   # samples (subjects)
n_regions = 8   # brain regions
n_time_points = 196  # time points in each fMRI time series

# Function to compute node features using Fourier Transform
def compute_frequency_domain_features(time_series):
    fft_features = np.abs(fft(time_series, axis=-1))
    return fft_features[:, :n_time_points // 2]

# Compute Fourier Transform features for each subject (node features)
node_features_list = [compute_frequency_domain_features(time_series[i]) for i in range(n_samples)]

# Function to compute correlation matrix (used as edges)
def compute_correlation_matrix(time_series):
    corr_matrix = np.corrcoef(time_series.T)  # Correlation matrix for brain regions
    return corr_matrix

correlation_matrices = [compute_correlation_matrix(time_series[i]) for i in range(n_samples)]

# Build graph data for each subject
graph_data_list = []
for i in range(n_samples):

    x = torch.tensor(node_features_list[i], dtype=torch.float)

    # Correlation matrix (edges)
    corr_matrix = correlation_matrices[i]
    edge_index = np.array(np.where(corr_matrix > 1.5))  # Edge threshold
    edge_index = torch.tensor(edge_index, dtype=torch.long)

    # Ensure valid edge indices
    num_nodes = x.size(0)
    edge_index = edge_index[:, (edge_index[0] < num_nodes) & (edge_index[1] < num_nodes)]

    # Create PyG Data object for the sample
    data = Data(x=x, edge_index=edge_index)
    graph_data_list.append(data)


train_idx, test_idx = train_test_split(np.arange(n_samples), test_size=0.2, random_state=0)

train_idx = train_idx.tolist() if isinstance(train_idx, torch.Tensor) else list(train_idx)
test_idx = test_idx.tolist() if isinstance(test_idx, torch.Tensor) else list(test_idx)

train_data = [graph_data_list[i] for i in train_idx]
train_labels = torch.tensor([ASD_labels[i] for i in train_idx], dtype=torch.long)
test_labels = torch.tensor([ASD_labels[i] for i in test_idx], dtype=torch.long)
test_data = [graph_data_list[i] for i in test_idx]

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the SparseGAT
class SparseGAT(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes, heads=8):
        super(SparseGAT, self).__init__()
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv3 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv4 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv5 = GATConv(hidden_channels * heads, num_classes, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the SparseGAT model
model = SparseGAT(num_node_features=node_features_list[0].shape[1], hidden_channels=64, num_classes=2, heads=8)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train():
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()

        out = model(batch)
        print(f'Output shape: {out.shape}')

        batch_labels = train_labels[batch.batch]
        print(f'Batch labels shape: {batch_labels.shape}')

        loss = F.nll_loss(out, batch_labels)
        loss.backward()
        optimizer.step()

# Test function
def test(loader, labels):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            out = model(batch)
            print(f'Output shape: {out.shape}')

            pred = out.argmax(dim=1)
            print(f'Pred shape: {pred.shape}')

            batch_indices = batch.batch
            batch_labels = labels[batch_indices]

            correct += (pred == batch_labels).sum().item()
            total += batch_labels.size(0)

    accuracy = correct / total if total > 0 else 0
    return accuracy * 100

# Training loop
for epoch in range(100):
    train()
    train_acc = test(train_loader, train_labels)
    test_acc = test(test_loader, test_labels)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
